# Testing neural networks approach to numerical integration on Genz numerical integration testing package 

These tests are dedicated to research of dependency of N size of dataset which is iused to train neural network. Also the tests will be done with both 

The tests will be done on 2D functions

## Library import

In [1]:
%run ../skuld/skuld.py      # skuld NNI library
%run ../utils/plots.py      # plotting functions
%run ../utils/integrate.py  # 'classic' numerical integration function

import pickle

# Part 1: N-size tests with Uniform Grid

## 2D testing function

In [2]:
def test_2d_integration(h, ej, func, func_float, a, b, func_name, logplot=False):
    global input_size, hidden_size, u, c, n_samples
  
    X_init, y_init = generate_data(func, a, b, n_samples, input_size, u, c)
    if logplot:
        plot_2d_function_heatmap_with_log(X_init, y_init, func_name)
    else:
        plot_2d_function_heatmap(X_init, y_init, func_name)
    X, y = scale_data(X_init, y_init, n_dim=2)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)  
    model = MLP(input_size, hidden_size)
    model.compile(criterion=nn.MSELoss(), optimizer=optim.Adam(model.parameters(), lr=learning_rate))
    train_history = model.fit(x_train, y_train, num_epochs, verbose=False)
    test_loss = model.test(x_test, y_test)
    print(f"Test Loss: {test_loss:.10f}")
    
    nni_scaled = NeuralNumericalIntegration.integrate(model, a, b, n_dims=2)
    nni_result = descale_result(float(nni_scaled[0]), X_init, y_init, frange=(0, 1), n_dim=2)
    result_quad = integrate_2d_nquad(func_float, a, b, u, c)
    result_trapz = integrate_2d_trapz(func, a, b, 100, u, c)
   
    return nni_result, result_quad, result_trapz

## Data operations functions

In [3]:
def save_data(data, filename="data.pkl"):
    with open(filename, 'wb') as f:
        pickle.dump(data, f)
    print(f"Data saved to {filename}")

def load_data(filename="data.pkl"):
    with open(filename, 'rb') as f:
        loaded_data = pickle.load(f)
    print(f"Data loaded from {filename}")
    
    return loaded_data

def calculate_mae(results):
    absolute_errors = [abs(a - b) for a, b, _ in results]
    mae = sum(absolute_errors) / len(absolute_errors)
    
    return mae

## Hyperparams for 2D

In [4]:
input_size = 2
hidden_size = 25
learning_rate = 0.001
num_epochs = 5000

init_n_samles = 10
n_samples = init_n_samles

tests_num = 20

a_ = [0.0, 0.0]
b_ = [1.0, 1.0]

all_results = []

## 1. Oscillatory 2D

In [ ]:
func_name = 'Oscillatory 2D'
h = 100
ej = 1
results = []

u_1 = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c_1 = [fraction * value for value in cs]

def osc_2d(X, u, c):
    sum_ = 0
    for i in range(input_size):
        sum_ += c[i] * X[:, i]

    return torch.cos(2 * math.pi * u[0] + sum_)
    
def osc_2d_float(X, u, c):
    sum_ = 0
    for i in range(input_size):
        sum_ += c[i] * X[i]
    return math.cos(2 * math.pi * u[0] + sum_)

for i in range(tests_num):
    n_samples = init_n_samles * (i + 1)
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples**2}\033[0m")
    u = u_1
    c = c_1
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))

print(h, ej, u_1, c_1)

plot_test_results(results)

all_results.append(results)

## 2. Product Peak 2D

In [ ]:
func_name = 'Product Peak 2D'
h = 150
ej = 1
results = []

u_2 = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c_2 = [fraction * value for value in cs]

def prod_peek_2d(X, u, c):
    prod_ = 1
    for i in range(2):
        prod_ *= (c[i] ** (-2) + (X[:, i] - u[i])**2) ** (-1)
    return prod_

def prod_peek_2d_float(X, u, c):
    prod_ = 1
    for i in range(2):
        prod_ *= (c[i] ** (-2) + (X[i] - u[i])**2) ** (-1)
    return prod_

for i in range(tests_num):
    n_samples = init_n_samles * (i + 1)
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples**2}\033[0m")
    u = u_2
    c = c_2
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))

print(h, ej, u_2, c_2)

plot_test_results(results)

all_results.append(results)

## 3. Corner Peak 2D

In [ ]:
func_name = 'Corner Peak 2D'
h = 600
ej = 1
results = []

u_3 = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c_3 = [fraction * value for value in cs]

def corn_peek_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] * X[:, i]
    return (1 + sum_) ** (-3)

def corn_peek_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] * X[i]
    return (1 + sum_) ** (-3)

for i in range(tests_num):
    n_samples = init_n_samles * (i + 1)
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples**2}\033[0m")
    u = u_3
    c = c_3
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))

print(h, ej, u_3, c_3)

plot_test_results(results)

all_results.append(results)

## 4. Gaussian 2D

In [ ]:
func_name = 'Gaussian 2D'
h = 150
ej = 1
results = []

u_4 = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c_4 = [fraction * value for value in cs] 

def gauss_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += (c[i] ** 2) * ((X[:, i] - u[i]) ** 2)
    return torch.exp(-sum_)

def gauss_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += (c[i] ** 2) * ((X[i] - u[i]) ** 2)
    return np.exp(-sum_)

for i in range(tests_num):
    n_samples = init_n_samles * (i + 1)
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples**2}\033[0m")
    u = u_4
    c = c_4
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))

print(h, ej, u_4, c_4)

plot_test_results(results)

all_results.append(results)

## 5. Continuous 2D

In [ ]:
func_name = 'Continuous 2D'
h = 100
ej = 1
results = []

u_5 = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c_5 = [fraction * value for value in cs] 

def cont_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i]  * abs(X[:, i] - u[i])
    return torch.exp(-sum_)

def cont_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] ** 2 * abs(X[i] - u[i]) ** 2
    return np.exp(-sum_)

for i in range(tests_num):
    n_samples = init_n_samles * (i + 1)
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples**2}\033[0m")
    u = u_5
    c = c_5
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))

print(h, ej, u_5, c_5)

plot_test_results(results)

all_results.append(results)

## 6. Discontinuous 2D

In [ ]:
func_name = 'Discontinuous 2D'
h = 16
ej = 1
results = []

u_6 = [random.uniform(0, 1) for _ in range(input_size)]
cs = [random.uniform(0, 1) for _ in range(input_size)]
fraction = h / (input_size ** ej * sum(cs))
c_6 = [fraction * value for value in cs]

def disco_2d(X, u, c):
    result = torch.zeros(X.shape[0])
    for i in range(X.shape[0]):
        x = X[i, 0].item()
        
        if x > u[0]:
            result[i] = 0.0
        else:
            result[i] = math.exp(c[0] * x)

    return result

def disco_1d_float(X, u, c):
    x1, x2 = X

    if x1 > u[0] or x2 > u[1]:
        return 0.0
    else:
        return math.exp(c[0] * x1 + c[1] * x2)

for i in range(tests_num):
    n_samples = init_n_samles * (i + 1)
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples**2}\033[0m")
    u = u_6
    c = c_6
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))

print(h, ej, u_6, c_6)

plot_test_results(results)

all_results.append(results)

# Calculation of MAEs for the research

In [11]:
ug_all_maes = []

for results in all_results:
    results_list = []
    chunk_size = 3
    for i in range(0, len(results), chunk_size):
        results_list.append(results[i:i + chunk_size])
    maes = [calculate_mae(results) for results in results_list]
    ug_all_maes.append(maes)

In [12]:
for maes in ug_all_maes:
    for i, mae in enumerate(maes, start=1):
        print(f"{mae:.8f}")
    print('\n')

0.02905872
0.01246002
0.00189221
0.00737003
0.00114893
0.00197969
0.00423909
0.00103979
0.00289963
0.00227505
0.00110572
0.00135106
0.00072779
0.00207903
0.00146991
0.00071857
0.00124436
0.00104918
0.00112905
0.00090732


5282.18550111
1118.33595276
1002.50988262
362.14988709
823.37937928
396.66124980
110.25701395
586.43468730
160.04634095
322.47573344
151.68341573
57.11209616
138.13588969
112.94347637
120.76736960
112.10463969
372.63657125
125.96398418
100.21821595
119.21987407


0.00429416
0.00250011
0.00112430
0.00077601
0.00058959
0.00041422
0.00038569
0.00025224
0.00087508
0.00016347
0.00024836
0.00022419
0.00016974
0.00019842
0.00011835
0.00012485
0.00011983
0.00008081
0.00009640
0.00008944


0.00331808
0.00119746
0.00053944
0.00001226
0.00013458
0.00021851
0.00016649
0.00009660
0.00006493
0.00014356
0.00007783
0.00035081
0.00010476
0.00007217
0.00007771
0.00007356
0.00005962
0.00002750
0.00006866
0.00009260


0.01122312
0.00576096
0.00572122
0.00484996
0.00305541
0.00152298
0.00

In [13]:
save_data(all_results, filename="cucumbers/NG-all-rests.pkl")
loaded_data = load_data(filename="cucumbers/NG-all-rests.pkl")

loaded_data == all_results

Data saved to NG-all-rests.pkl
Data loaded from NG-all-rests.pkl


True

In [14]:
save_data(ug_all_maes, filename="cucumbers/NG-all-maes.pkl")
loaded_data = load_data(filename="cucumbers/NG-all-maes.pkl")

loaded_data == ug_all_maes

Data saved to NG-all-maes.pkl
Data loaded from NG-all-maes.pkl


True

# Part 2: N-size tests with Standard Uniform Distribution

## 2D testing function

In [15]:
def test_2d_integration(h, ej, func, func_float, a, b, func_name, logplot=False):
    global input_size, hidden_size, u, c, n_samples
  
    X_init, y_init = generate_data_uniform(func, a, b, n_samples, input_size, u, c)
    if logplot:
        plot_2d_function_heatmap_with_log(X_init, y_init, func_name)
    else:
        plot_2d_function_heatmap(X_init, y_init, func_name)
    X, y = scale_data(X_init, y_init, n_dim=2)
    
    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1)  
    model = MLP(input_size, hidden_size)
    model.compile(criterion=nn.MSELoss(), optimizer=optim.Adam(model.parameters(), lr=learning_rate))
    train_history = model.fit(x_train, y_train, num_epochs, verbose=False)
    test_loss = model.test(x_test, y_test)
    print(f"Test Loss: {test_loss:.10f}")
    
    nni_scaled = NeuralNumericalIntegration.integrate(model, a, b, n_dims=2)
    nni_result = descale_result(float(nni_scaled[0]), X_init, y_init, frange=(0, 1), n_dim=2)
    result_quad = integrate_2d_nquad(func_float, a, b, u, c)
    result_trapz = integrate_2d_trapz(func, a, b, 100, u, c)
   
    return nni_result, result_quad, result_trapz

## Hyperparams for 2D

In [16]:
input_size = 2
hidden_size = 25
learning_rate = 0.001
num_epochs = 5000

init_n_samles = 10
n_samples = init_n_samles

tests_num = 20

a_ = [0.0, 0.0]
b_ = [1.0, 1.0]

all_results = []

## 1. Oscillatory 2D

In [ ]:
func_name = 'Oscillatory 2D'
h = 100
ej = 1
results = []

def osc_2d(X, u, c):
    sum_ = 0
    for i in range(input_size):
        sum_ += c[i] * X[:, i]

    return torch.cos(2 * math.pi * u[0] + sum_)
    
def osc_2d_float(X, u, c):
    sum_ = 0
    for i in range(input_size):
        sum_ += c[i] * X[i]
    return math.cos(2 * math.pi * u[0] + sum_)

for i in range(tests_num):
    n_samples = (init_n_samles * (i + 1)) ** 2
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples}\033[0m")
    u = u_1
    c = c_1
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))
    results.append(test_2d_integration(h, ej, osc_2d, osc_2d_float, a_, b_, func_name))

print(h, ej, u_1, c_1)

plot_test_results(results)

all_results.append(results)

## 2. Product Peak 2D

In [ ]:
func_name = 'Product Peak 2D'
h = 150
ej = 1
results = []

def prod_peek_2d(X, u, c):
    prod_ = 1
    for i in range(2):
        prod_ *= (c[i] ** (-2) + (X[:, i] - u[i])**2) ** (-1)
    return prod_

def prod_peek_2d_float(X, u, c):
    prod_ = 1
    for i in range(2):
        prod_ *= (c[i] ** (-2) + (X[i] - u[i])**2) ** (-1)
    return prod_

for i in range(tests_num):
    n_samples = (init_n_samles * (i + 1)) ** 2
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples}\033[0m")
    u = u_2
    c = c_2
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, prod_peek_2d, prod_peek_2d_float, a_, b_, func_name, logplot=False))

print(h, ej, u_2, c_2)

plot_test_results(results)

all_results.append(results)

## 3. Corner Peak 2D

In [ ]:
func_name = 'Corner Peak 2D'
h = 600
ej = 1
results = []

def corn_peek_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] * X[:, i]
    return (1 + sum_) ** (-3)

def corn_peek_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] * X[i]
    return (1 + sum_) ** (-3)

for i in range(tests_num):
    n_samples = (init_n_samles * (i + 1)) ** 2
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples}\033[0m")
    u = u_3
    c = c_3
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, corn_peek_2d, corn_peek_2d_float, a_, b_, func_name, logplot=True))

print(h, ej, u_3, c_3)

plot_test_results(results)

all_results.append(results)

## 4. Gaussian 2D

In [ ]:
func_name = 'Gaussian 2D'
h = 150
ej = 1
results = []

def gauss_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += (c[i] ** 2) * ((X[:, i] - u[i]) ** 2)
    return torch.exp(-sum_)

def gauss_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += (c[i] ** 2) * ((X[i] - u[i]) ** 2)
    return np.exp(-sum_)

for i in range(tests_num):
    n_samples = (init_n_samles * (i + 1)) ** 2
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples}\033[0m")
    u = u_4
    c = c_4
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))
    results.append(test_2d_integration(h, ej, gauss_2d, gauss_2d_float, a_, b_, func_name, logplot=True))

print(h, ej, u_4, c_4)

plot_test_results(results)

all_results.append(results)

## 5. Continuous 2D

In [ ]:
func_name = 'Continuous 2D'
h = 100
ej = 1
results = []

def cont_2d(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i]  * abs(X[:, i] - u[i])
    return torch.exp(-sum_)

def cont_2d_float(X, u, c):
    sum_ = 0
    for i in range(2):
        sum_ += c[i] ** 2 * abs(X[i] - u[i]) ** 2
    return np.exp(-sum_)

for i in range(tests_num):
    n_samples = (init_n_samles * (i + 1)) ** 2
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples}\033[0m")
    u = u_5
    c = c_5
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, cont_2d, cont_2d_float, a_, b_, func_name, logplot=False))

print(h, ej, u_5, c_5)

plot_test_results(results)

all_results.append(results)

## 6. Discontinuous 2D

In [ ]:
func_name = 'Discontinuous 2D'
h = 16
ej = 1
results = []

def disco_2d(X, u, c):
    result = torch.zeros(X.shape[0])
    for i in range(X.shape[0]):
        x = X[i, 0].item()
        
        if x > u[0]:
            result[i] = 0.0
        else:
            result[i] = math.exp(c[0] * x)

    return result

def disco_1d_float(X, u, c):
    x1, x2 = X

    if x1 > u[0] or x2 > u[1]:
        return 0.0
    else:
        return math.exp(c[0] * x1 + c[1] * x2)

for i in range(tests_num):
    n_samples = (init_n_samles * (i + 1)) ** 2
    print(f"\n\033[1mIteration {i+1} is running! N = {n_samples}\033[0m")
    u = u_6
    c = c_6
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))
    results.append(test_2d_integration(h, ej, disco_2d, disco_1d_float, a_, b_, func_name, logplot=False))

print(h, ej, u_6, c_6)

plot_test_results(results)

all_results.append(results)

# Calculation of MAEs for the research

In [23]:
sud_all_maes = []

for results in all_results:
    results_list = []
    chunk_size = 3
    for i in range(0, len(results), chunk_size):
        results_list.append(results[i:i + chunk_size])
    maes = [calculate_mae(results) for results in results_list]
    sud_all_maes.append(maes)

In [24]:
for maes in sud_all_maes:
    for i, mae in enumerate(maes, start=1):
        print(f"{mae:.8f}")
    print('\n')

0.02864941
0.01681226
0.04003308
0.01261231
0.00538154
0.00365728
0.00888137
0.00602912
0.00306726
0.00533309
0.00307636
0.00302148
0.00555016
0.00438010
0.00133525
0.00485473
0.00372891
0.00216722
0.00454841
0.00578191


6049.95693461
4303.86931864
995.83462016
1307.51463318
680.27407329
814.28562928
1198.73289490
469.53207906
1135.90740458
368.95691426
329.22589620
313.86118062
355.85255432
522.76398214
289.50375875
245.73484803
350.29767355
94.42625428
250.98650614
129.52332051


0.00006646
0.00004483
0.00008672
0.00003302
0.00006534
0.00002935
0.00003172
0.00002290
0.00001646
0.00003397
0.00001837
0.00002346
0.00002601
0.00003260
0.00001782
0.00000398
0.00001049
0.00001225
0.00001037
0.00002798


0.00255228
0.00238211
0.00148250
0.00079782
0.00083486
0.00043834
0.00010016
0.00067836
0.00025422
0.00047908
0.00034066
0.00020877
0.00039998
0.00019528
0.00018979
0.00020490
0.00026298
0.00005977
0.00016693
0.00019052


0.00679688
0.00291698
0.00502197
0.00072248
0.00164660
0.00205485
0.

In [25]:
save_data(all_results, filename="cucumbers/SUD-all-rests.pkl")
loaded_data = load_data(filename="cucumbers/SUD-all-rests.pkl")

loaded_data == all_results

Data saved to SUD-all-rests.pkl
Data loaded from SUD-all-rests.pkl


True

In [26]:
save_data(sud_all_maes, filename="cucumbers/SUD-all-maes.pkl")
loaded_data = load_data(filename="cucumbers/SUD-all-maes.pkl")

loaded_data == sud_all_maes

Data saved to SUD-all-maes.pkl
Data loaded from SUD-all-maes.pkl


True